# Práctica 1: Clasificación multi-etiqueta

## Ejercicio 6

Repasa lo que ya estudiado anteriormente sobre validación cruzada, investiga las funciones:
- KFold() y cross_validate(), cross_val_score y make_scorer(). 
- ¿Cómo aplicarías validación cruzada en el contexto ML? Pega a continuación el código de un ejemplo de uso.

### Enfoque

Para realizar este apartado se va a tomar el conjunto de datos `scene` ya que ha sido uno de los conjuntos de datos que menos tiempo ha llevado procesar en los apartados anteriores, siendo un conjunto de datos no muy grande pero tampoco pequeño. Lo que lo hace ideal para realizar pruebas sin que lleven un tiempo excesivo. Sus características, obtenidas en apartados anteriores, son las siguientes:

- Número de instancias en scene: 2407
- Número de atributos en scene: 294
- Número de etiquetas en scene: 6
- Car en scene: 1.0739509763190693
- Den en scene: 0.17899182938651156
- Div en scene: 0.234375
- AvgIR en scene: 1.2537840597429668
- rDep en scene: 0.9333333333333333

In [5]:
# Actualizamos a ultima versión de nuevo
#!pip uninstall scikit-learn -y
#!pip install scikit-learn

### Validación cruzada

La validación cruzada es una técnica utilizada en Machine Learning para evaluar el rendimiento de un modelo estadístico. La idea es dividir el conjunto de datos en subconjuntos de entrenamiento y prueba de manera repetida, de modo que el modelo se entrene y evalúe en diferentes subconjuntos. Esto ayuda a obtener una medida más robusta y fiable del rendimiento del modelo al considerar múltiples divisiones de los datos. Evitando así caer en evaluaciones demasiado optimistas o pesimistas por una divisón poco independiente del conjunto de datos. Es lo que en inglés se conoce como *bias*, es decir, crear un modelo poco imparcial, con tendencias debido al conjunto de datos.

En el contexto de scikit-learn, hay varias funciones y clases relacionadas con la validación cruzada. Aquí te explico algunas de ellas:
- `KFold()`: Esta clase divide el conjunto de datos en k pliegues (folds) diferentes para realizar la validación cruzada. Cada fold se utiliza como conjunto de prueba exactamente una vez.
- `cross_validate()`: Esta función realiza validación cruzada y devuelve medidas de rendimiento (como precisión, recall, tiempo de entrenamiento, etc.) para cada repetición de la validación cruzada.
- `cross_val_score()`: Esta función calcula la puntuación de rendimiento para cada repetición de la validación cruzada y devuelve una matriz de resultados.
- `make_scorer()`: Esta función crea un objeto de puntuación a partir de una función de puntuación personalizada, lo que facilita su uso en funciones de evaluación de scikit-learn.

Durante diferentes pruebas se han encontrado algunos problemas, por lo que no se usarán las siguientes funciones:
- `make_scorer()`: Requiere el atributo en el clasificador `_classes`, sin embargo, los clasificadores de scikit-multilearn no tienen este atributo por lo que no se puede utilizar con estos algoritmos. La alternativa es calcular las métricas que se quieren usar independientemente.

Por otro lado `cross_validate()` y `cross_val_score()` realizan la validación cruzada de forma "automática" mientras que la función `KFold()` la realiza de forma manual. Para un mejor entendimiento de la validación cruzada se va a realizar de forma manual, con `KFold()`. Así se tiene mayor control sobre el código y los resultados permitiendo realizar experimentaciones más detalladas y avanzadas. Entre otras razones, `cross_val_score()` no devuelve los modelos, por lo que limita la experimentación si se quiere experimentar con el modelo que peor rendimiento ha obtenido, o se quiere experimentar con uno que haya obtenido muy buenos resultados (o malos) en una métrica en concreto. `cross_validate()` parece ser más flexible en cuanto a la especificación de métricas de interés, sin embargo, sigue sin devolver todos los modelos y predicciones. Como esta práctica consiste en afianzar los conocimientos de multilabel y las diferentes funciones y posibilidades de las librerías de scikit y scikitmultilearn se va a utilizar la que nos da mayor flexibilidad y control sobre los resultados que se obtengan.

In [6]:
from skmultilearn.dataset import load_dataset
from sklearn.model_selection import KFold
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Scikit-multilearn permite importar los conjuntos de datos ya divididos en conjuntos de entrenamiento y test, pero como queremos probar cómo funciona kfold vamos a cargar el conjunto de datos completo.

In [7]:
X, y, _, _ = load_dataset('scene', 'undivided')

scene:undivided - exists, not redownloading


A continuación se muestra el caso de uso de validación cruzada con `KFolds`.

Se ha decidido utilizar una k de 20 para diversificar sin poner un número muy elevado, ya que podría traducirse en tiempo computacional muy alto.

El Clasificador elegido es uno de los vistos en el apartado anterior, `LabelPowerSet`, no hay ninguna razón en específico por la que se haya decidido usar este. Pero una justificación principal es que, los métodos de transformación no han dado ningún problema con la versión de la librería. Y entre los métodos de transformación fue el que mejor accuracy consiguió en el dataset scene en el apartado anterior.

Lo que se va a realizar es iterativamente entrenamietno en el clasificador con los diferentes folds, y se irán guardando todos los datos. Al finalizar se mostrará el que haya mostrado los mejores resultados. Como sólo se quiere ver cómo funciona `KFolds` no se va a realizar una experimentación muy profunda, sólo se valorará la métrica accuracy. Pero las posibilidades son mucho más extensas. Por otro lado, nos quedaremos con el clasificador que mejor accuracy consiga.

In [9]:
# Número de folds
k = 20
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Clasificador
base_classifier = GaussianNB()
classifier = LabelPowerset(base_classifier)

# Listas de resultados inicializadas
accuracies = []
classifiers = []
predictions = []
X_trains = []
X_tests = []
y_trains = []
y_tests = []

# Iterar sobre los folds
for train_index, test_index in kf.split(X):

    # Split the data into training and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Guardar subconjuntos
    X_trains.append(X_train)
    X_tests.append(X_test)
    y_trains.append(y_train)
    y_tests.append(y_test)

    # Entrenar clasificador
    classifier.fit(X_train, y_train)
    classifiers.append(classifier)

    # Predicciones
    prediction = classifier.predict(X_test)
    predictions.append(prediction)

    # Accuracy
    accuracy = accuracy_score(y_test, prediction)
    accuracies.append(accuracy)

# Mostrar mejor clasificador
index = accuracies.index(max(accuracies))
best_classifier = classifiers[index]
best_prediction = predictions[index]
best_X_train = X_trains[index]
best_X_test = X_tests[index]
best_y_train = y_trains[index]
best_y_test = y_tests[index]

print("The accuracy for each fold is : " + str(accuracies))
print("The best accuracy is : " + str(max(accuracies)))
print("The best classifier is the trained with the fold number: " + str(index))

The accuracy for each fold is : [0.5702479338842975, 0.5950413223140496, 0.5619834710743802, 0.6115702479338843, 0.5454545454545454, 0.6033057851239669, 0.49586776859504134, 0.575, 0.6083333333333333, 0.5083333333333333, 0.43333333333333335, 0.5333333333333333, 0.6, 0.6, 0.5333333333333333, 0.525, 0.4666666666666667, 0.5666666666666667, 0.5083333333333333, 0.49166666666666664]
The best accuracy is : 0.6115702479338843
The best classifier is the trained with the fold number: 3


En este apartado se ha aprendido a implementar la validación cruzada en un problema de clasificación multietiqueta y las posibilidades de validación cruzada que existen con la librería scikit. 

La validación cruzada es una técnica de evaluación de modelos que permite obtener una medida más robusta del rendimiento del modelo. En este caso de uso realizado, en lugar de dividir el conjunto de datos en un único conjunto de entrenamiento y prueba, la validación cruzada divide el conjunto de datos en 'k' folds y entrena y evalúa el modelo 'k' veces, cada vez con un subconjunto diferente como conjunto de prueba.